In [1]:
from datetime import datetime, timedelta

import pandas as pd 
import altair as alt
import numpy as np
import pytz

## Lectura de los archivos de los hilos

Los archivos se dividen en dos grupos principales:

 - El archivo *threads.csv* y,
 - Los archivos *comments/comments_[THREAD_ID].csv*.

El primer grupo es un único archivo que contiene información de alto nivel que actúa como agregador para el resto de los archivos. En este archivo se puede encontrar el nombre, el autor, el título, la fecha de creación, la puntuación y el número de comentarios de cada uno de los *hilos vivos* relacionados con la invasión.

In [2]:
threads = pd.read_csv('data/threads.csv')
threads.head(2)

,id,name,author,title,created_utc,created_at,num_comments,score,upvote_ratio,permalink
0,ssn7ic,t3_ssn7ic,09cb3c204828ede2196452cf1fe87c59,r/WorldNews Live Thread: Ukraine-Russia Tensions,1.644878e+09,2022-02-14 22:35:01,6774,2207,0.92,/r/worldnews/comments/ssn7ic/rworldnews_live_t...
1,st8lq0,t3_st8lq0,6fb4eee62d702c106897cbed2f56feea,r/worldnews Live Thread: Ukraine-Russia Tensions,1.644947e+09,2022-02-15 17:43:15,8915,3595,0.92,/r/worldnews/comments/st8lq0/rworldnews_live_t...


El segundo grupo contiene tantos archivos como hilos existan en el archivo *threads.csv*; cada uno tiene un nombre como *comments/comments_[THREAD_ID].csv*.

Cada fila de estos archivos *csv* representa un comentario realizado en el hilo principal. La información disponible para cada comentario es: autor, identificador, cuerpo, fecha/hora de creación, si ha sido editado, puntuación y el comentario padre (si es una respuesta).

Una cosa a tener en cuenta es que no se puede utilizar simplemente `pd.read_csv`, ya que a veces los comentarios pueden contener saltos de línea que hacen que a veces un solo comentario utilice más de una fila del fichero. Para leer con éxito todos estos archivos, uno necesita pasar el argumento `lineterminator`:

In [6]:
comments = pd.read_csv('data/comments/comments__st8lq0.csv', lineterminator='\n')
comments.head(2)

,author,submission_id,id,body,edited,created_utc,link_id,parent_id,distinguished,depth,ups,downs,score,total_awards_received,gilded,gildings
0,160a3b6484fe752a9f15e1fff3d5b3d0,st8lq0,hx2f8f5,"""The State Emergency Service of Ukraine is pre...",False,1.644950e+09,t3_st8lq0,t3_st8lq0,NaN,0,190,0,190,0,0,NaN
1,0a8bc5553309fc8f0c301cd3d9f8a155,st8lq0,hx2hpq3,"MOSCOW, Feb 15 (Reuters) - President Vladimir ...",False,1.644951e+09,t3_st8lq0,t3_st8lq0,NaN,0,156,0,156,0,0,NaN


## Trazar la frecuencia de los comentarios

Ahora que hemos aprendido lo que contienen los archivos y cómo leerlos, vamos a hacer algo interesante con ellos. Veamos cómo ha cambiado el interés en el hilo a lo largo del tiempo contando el número de comentarios por hora.

El proceso general es el siguiente

 1. Leer todas las fechas de los comentarios
 2. Poner las fechas en intervalos de 1 hora
 3. Trazar.

### 1. Leer todas las fechas de los comentarios

Podemos seguir usando *pandas* pero siendo más inteligentes en su uso. ¿Sabías que puedes especificar que sólo quieres un subconjunto de columnas con el argumento `usecols`?

In [7]:
create_dates = []

for thread_id in threads['id']:
    data = pd.read_csv(f'data/comments/comments__{thread_id}.csv', lineterminator='\n', usecols=['created_utc'])
    create_dates.append(data['created_utc'].values)

create_dates = np.concatenate(create_dates)

Esto nos deja con la matriz NumPy `created_dates` que contiene 2.083.085$ números que representan la fecha de creación de cada comentario. El siguiente paso es dividir estas horas en intervalos de 1 hora.

### 2. Redondear los tiempos de creación

Utilizaremos un par de funciones de ayuda para redondear las horas de las fechas hacia arriba o hacia abajo al paso más cercano en el intervalo que definamos (y una más para visualizar las marcas de tiempo).

In [9]:
# Funciones de ayuda a la fecha

INTERVAL = 3600# 1 hora en segundos

def lower_bound(ts):
    return ts - (ts % INTERVAL)

def upper_bound(ts):
    return ts + (INTERVAL - ((ts) % INTERVAL) if (ts) % INTERVAL != 0 else 0)

def humanise(ts):
    return datetime.fromtimestamp(ts).strftime('%m/%d/%Y, %H:%M:%S')

Por ejemplo, toma la fecha *04/29/2022, 20:20:58* cuyo timestamp es `1651263658`:

In [10]:
example_ts = 1651263658

actual_date = humanise(example_ts)
upper = humanise(upper_bound(example_ts))
lower = humanise(lower_bound(example_ts))

print(f'{lower} es el límite inferior de {actual_date} y su límite superior es {upper}')

04/29/2022, 15:00:00 es el límite inferior de 04/29/2022, 15:20:58 y su límite superior es 04/29/2022, 16:00:00


Ahora que tenemos una forma de calcular los límites superior e inferior de una fecha específica, podemos pasar a calcular los bordes de la bandeja. Esto es fácil una vez que conocemos las fechas mínimas y máximas en nuestro `created_dates`. De hecho, la obtención de los bordes de la bandeja es una línea de uno con NumPy: